<div align=justify dir=rtl>

<p>
در این بخش قرار است به کمک مدل DINO که قبلا روی حجم بسیاز زیادی داداه آموزش دیده کار کنیم و به ازای هر تصویر بردار attention آنها را ترسیم میکنیم و محاسبه میکنیم که اگر تعداد پیکسل هایی که میزان attention در آنها از یک حد آستانه ای  بیشتر باشد از یک مقدار مشخص بالا تر بود ،  آن تصویر را به عنوان تصویر مناسب در نظر میگیریم
</p>


<div align=justify dir=rtl>
<h2>بارگذاری داده</h2>
<p>
اگر بر روی کولب کار میکنیم در ابتدا لازم است یک سری کتابحانه نصب گردند
</p>


In [ ]:
image_lists=[]
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO
z=0
import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits

<div align=justify dir=rtl>
<h2>محاسبه میزان attention و فیلتر تصاویر</h2>
<p>
همانند بخش آموزش مدل DINO در اینجا نیز یک سری آرگومان داریم که با قرار دادن مقدار default با مقداری که مدنظرمان است آنها را پر میکنیم.
</p>
<ul>
<li>
arch: با این آرگومان مشخص میکنیم که بک بون مدل چه چیز است.
</li>
<li>
patch_size : سایز وصله های ورودی مدل را مشخص میکنیم

</li>
<li>
pretrained_weights : مشخص کننده مسیر وزن های مدل أموزش دیده
</li>
</ul>

</div>

In [ ]:

if __name__ == '__main__':
    parser = argparse.ArgumentParser('Visualize Self-Attention maps')
    parser.add_argument('--arch', default='vit_small', type=str,
        choices=['vit_tiny', 'vit_small', 'vit_base'], help='Architecture (support only ViT atm).')
    parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument('--pretrained_weights', default='vits_tcga_brca_dino.pt', type=str,
        help="Path to pretrained weights to load.")
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    parser.add_argument("--image_path", default="", type=str, help="")
    parser.add_argument("--image_size", default=(224, 224), type=int, nargs="+", help="Resize image.")
    parser.add_argument('--output_dir', default='/content/', help='Path where to save visualizations.')
    parser.add_argument("--threshold", type=float, default=None, help="""We visualize masks
        obtained by thresholding the self-attention maps to keep xx% of the mass.""")
    args = parser.parse_args("")

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    # build model
    model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
    for p in model.parameters():
        p.requires_grad = False
    model.eval()
    model.to(device)
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    else:
        print("Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.")
        url = None
        if args.arch == "vit_small" and args.patch_size == 16:
            url = "dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth"
        elif args.arch == "vit_small" and args.patch_size == 8:
            url = "dino_deitsmall8_300ep_pretrain/dino_deitsmall8_300ep_pretrain.pth"  # model used for visualizations in our paper
        elif args.arch == "vit_base" and args.patch_size == 16:
            url = "dino_vitbase16_pretrain/dino_vitbase16_pretrain.pth"
        elif args.arch == "vit_base" and args.patch_size == 8:
            url = "dino_vitbase8_pretrain/dino_vitbase8_pretrain.pth"
        if url is not None:
            print("Since no pretrained weights have been provided, we load the reference pretrained DINO weights.")
            state_dict = torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/" + url)
            model.load_state_dict(state_dict, strict=True)
        else:
            print("There is no reference weights available for this model => We use random weights.")

    # open image
    import os
    path = 'E:/NLP/content/normalized/'
    folders = os.listdir(path)
    for folder in folders:
      images=path+folder+'/1024/'
      images = os.listdir(images)
      # print(os.listdir(images))
      # break
      for image in images:
        image_path = path+folder+'/1024/'+image
        if image_path is None:
            # user has not specified any image - we use our own image
            print("Please use the `--image_path` argument to indicate the path of the image you wish to visualize.")
            print("Since no image path have been provided, we take the first image in our paper.")
            response = requests.get("https://dl.fbaipublicfiles.com/dino/img.png")
            img = Image.open(image_path)
            img = img.convert('RGB')
        elif os.path.isfile(image_path):
            with open(image_path, 'rb') as f:
                img = Image.open(f)
                img = img.convert('RGB')
        else:
            print(f"Provided image path {image_path} is non valid.")
            sys.exit(1)
        transform = pth_transforms.Compose([
            pth_transforms.Resize(args.image_size),
            pth_transforms.ToTensor(),
            pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
        img = transform(img)

        # make the image divisible by the patch size
        w, h = img.shape[1] - img.shape[1] % args.patch_size, img.shape[2] - img.shape[2] % args.patch_size
        img = img[:, :w, :h].unsqueeze(0)

        w_featmap = img.shape[-2] // args.patch_size
        h_featmap = img.shape[-1] // args.patch_size

        attentions = model.get_last_selfattention(img.to(device))

        nh = attentions.shape[1] # number of head

        # we keep only the output patch attention
        attentions = attentions[0, :, 0, 1:].reshape(nh, -1)

        if args.threshold is not None:
            # we keep only a certain percentage of the mass
            val, idx = torch.sort(attentions)
            val /= torch.sum(val, dim=1, keepdim=True)
            cumval = torch.cumsum(val, dim=1)
            th_attn = cumval > (1 - args.threshold)
            idx2 = torch.argsort(idx)
            for head in range(nh):
                th_attn[head] = th_attn[head][idx2[head]]
            th_attn = th_attn.reshape(nh, w_featmap, h_featmap).float()
            # interpolate
            th_attn = nn.functional.interpolate(th_attn.unsqueeze(0), scale_factor=args.patch_size, mode="nearest")[0].cpu().numpy()

        attentions = attentions.reshape(nh, w_featmap, h_featmap)
        attentions = nn.functional.interpolate(attentions.unsqueeze(0), scale_factor=args.patch_size, mode="nearest")[0].cpu().numpy()
        count1 = np.count_nonzero(attentions[0] > 0.01)
        count2 = np.count_nonzero(attentions[1] > 0.01)
        count3 = np.count_nonzero(attentions[2] > 0.01)


        # calculate the percentage of values greater than 0.01
        percentage1 = count1 / attentions[0].size
        percentage2 = count2 / attentions[1].size
        percentage3 = count3 / attentions[2].size

        # check if 80% of the values are greater than 0.01
        if percentage1 >= 0.1 and percentage2>=0.1 and percentage3 >=0.1:
        # if percentage1 >= 0.1 :
          
        #   print(image_path)
          image_lists.append(image_path)


## captions for images

In [ ]:
import os
import glob

folder_path = "E:/NLP/Finalized_samples 2"  # replace with the path to your folder

# get a list of all txt files in the folder
txt_files = glob.glob(os.path.join(folder_path, "*.txt"))

# create an empty dictionary to store file contents
file_contents = {}

# loop through each txt file
for txt_file in txt_files:
    # get the filename without the path and extension
    file_name = os.path.splitext(os.path.basename(txt_file))[0]
    
    # read the file content and store it in the dictionary
    with open(txt_file, "r") as f:
        content = f.read()
        file_contents[file_name] = content

captions=[]
for image in image_lists:
    captions.append(file_contents[image[26:38]])

<div align=justify dir=rtl>

<p>
در آخر تصاویر انتخابی را در قالب یک فایل csv ذخیره میکنیم تا در مراحل بعدی بتوانیم از آن استفاده کنیم
</p>

In [ ]:
import pandas as pd
df=pd.DataFrame()

df['images']=image_lists
df['captions']=captions
df.to_csv('filter_data.csv')